In [19]:
# importing libray to handle status bars
from tqdm.notebook import tqdm

# import libray to ignore warnings
import warnings
warnings.filterwarnings("ignore")

# importing library to deal with numeric arrays
import numpy as np

# importing deep learning library
import tensorflow as tf

#importing library to read files
import pickle

In [18]:
def img_feature_extraction(x_values, pre_model):

    data = []
    
    # preprocessing and then using pretrained neural nets to extract features to be fed into Global Pooling
    for image in tqdm(x_values):
        im_toarray = tf.keras.preprocessing.image.img_to_array(image)
        
        im_toarray = np.expand_dims(image, axis=0)
        im_toarray = tf.keras.applications.mobilenet_v2.preprocess_input(im_toarray)
        #im_toarray = tf.keras.applications.xception.preprocess_input(im_toarray)
        
        data.append(im_toarray)
        
    data_stack = np.vstack(data) 
    
    features = pre_model.predict(data_stack, batch_size=52)
    
    return data_stack, features

In [15]:
IMAGE_DIMS = (240, 240, 3)

In [16]:

xcp_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_DIMS, include_top=False, weights='imagenet')
    
for layer in xcp_model.layers:
    layer.trainable = False

In [ ]:
xcp_model.load_weights('xception.h5')

In [7]:
with open('train_X.pkl', 'rb') as f:
    train_X = pickle.load(f)
with open('val_y.pkl', 'rb') as f:
    train_y = pickle.load(f)

In [8]:
with open('test_X.pkl', 'rb') as f:
    test_X = pickle.load(f)
with open('test_y.pkl', 'rb') as f:
    test_y = pickle.load(f)

In [9]:
with open('val_X.pkl', 'rb') as f:
    val_X = pickle.load(f)
with open('val_y.pkl', 'rb') as f:
    val_y = pickle.load(f)

In [10]:
frames_val = [3546, 3413]
frames_test = [4832, 4326]
frames_train = [6388, 3620, 3000, 2938, 3220, 3908, 1645, 4692, 5736, 5064, 4720, 2916, 2597, 4739, 3653, 3612, 4678]

In [26]:
def extract(frames, model, feature_data, name):

    start = 0
    for i in range(len(frames)):
        if i > -1:
            if start == 0:
                p, f = img_feature_extraction(feature_data[ :frames[i]], model)

                
            else:
                p, f = img_feature_extraction(feature_data[start:start+frames[i]], model)



            with open(f'{name}_extract{i}.pkl', 'wb') as file:  # open a text file
                pickle.dump(f, file) 
            """with open(f'{name}_extract_proc{i}.pkl', 'wb') as file:  # open a text file
                pickle.dump(p, file) """

        
        start += frames[i]


In [39]:

extract(frames_test, xcp_model, test_X, 'test')


  0%|          | 0/3546 [00:00<?, ?it/s]

MemoryError: Unable to allocate 2.28 GiB for an array with shape (3546, 240, 240, 3) and data type float32

In [27]:
extract(frames_val, xcp_model, val_X, 'val')

  0%|          | 0/4832 [00:00<?, ?it/s]

93/93 [==============================] - 296s 3s/step


  0%|          | 0/4326 [00:00<?, ?it/s]

84/84 [==============================] - 275s 3s/step


In [16]:
extract(frames_train, xcp_model, train_X, 'train')

NameError: name 'train_X' is not defined